<p style="font-family: 'Courier New', Courier, monospace; font-size: 40px; font-weight: bold; color: blue;  text-align: center;">
  LoRaWAN Path Loss Measurements in an Indoor Setting: DATA QUERYING from InfluxdB
</p>

In [2]:
# Libraries/Packages Used:
import os  # For accessing environment variables
from dotenv import load_dotenv  # To load environment variables from a .env file
import pandas as pd  # For data manipulation and handling timezones
from influxdb import InfluxDBClient  # To interact with InfluxDB, a time-series database
import time  # For sleep function between batches
import pathlib  # For checking if file exists

In [3]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve the variables
host = os.getenv('INFLUXDB_HOST')
port = int(os.getenv('INFLUXDB_PORT'))  # Ensure port is an integer
database = os.getenv('INFLUXDB_DATABASE')

In [4]:
def fetch_data(start_time, end_time):
    """
    Fetch sensor data from InfluxDB between specified start and end times.

    Converts input times from Europe/Berlin to UTC for querying, then back to Europe/Berlin for use.
    Returns data as a pandas DataFrame for analysis.
    """
    # Initialize the InfluxDB client
    client = InfluxDBClient(host=host, port=port)
    client.switch_database(database)

    # Convert input times (Europe/Berlin) to UTC for the query
    start_time_utc = pd.to_datetime(start_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_utc = pd.to_datetime(end_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')

    # Query with the provided start_time and end_time in UTC
    # Use '>' for start_time to exclude the last fetched timestamp
    query = f"SELECT * FROM sensor_data WHERE time > '{start_time_utc}' AND time <= '{end_time_utc}'"

    result = client.query(query)
    df = pd.DataFrame(list(result.get_points()))

    if not df.empty:
        # Convert 'time' column to datetime with utc=True
        df['time'] = pd.to_datetime(df['time'], utc=True).dt.tz_convert('Europe/Berlin')

    return df

In [5]:
def fetch_data_in_batches(start_time, end_time):
    """
    Fetch sensor data in batches of 10 days with a 2-minute  break between each batch.
    """
    # Convert start_time to a datetime object
    start_time = pd.to_datetime(start_time)

    # Check if start_time is timezone-naive; if so, localize it
    if start_time.tzinfo is None:
        start_time = start_time.tz_localize('Europe/Berlin')
    else:
        start_time = start_time.tz_convert('Europe/Berlin')

    # Check if end_time is timezone-naive; if so, localize it
    end_time = pd.to_datetime(end_time)
    if end_time.tzinfo is None:
        end_time = end_time.tz_localize('Europe/Berlin')
    else:
        end_time = end_time.tz_convert('Europe/Berlin')

    # Initialize a list to store DataFrames
    df_list = []

    current_start = start_time
    delta = pd.Timedelta(days=10)

    while current_start < end_time:
        current_end = min(current_start + delta, end_time)

        print(f"Fetching data from {current_start} to {current_end}")

        # Fetch data for the current interval
        df = fetch_data(current_start.strftime('%Y-%m-%d %H:%M:%S'), current_end.strftime('%Y-%m-%d %H:%M:%S'))

        if not df.empty:
            df_list.append(df)

        # Sleep for 30 seconds between queries
        if current_end < end_time:
            print("Sleeping for 2 minutes...")
            time.sleep(120)

        # Move to the next interval
        current_start = current_end

    # Combine all DataFrames
    if df_list:
        batch_combined_df = pd.concat(df_list, ignore_index=True)
    else:
        batch_combined_df = pd.DataFrame()

    return batch_combined_df

In [6]:
# File path for the combined data CSV
csv_file_path = '../all_data_files/combined_measurements_data.csv'

# Starting campaign time
initial_start_time = '2024-09-26 13:00:00'  # Berlin time

# Check if the CSV file exists
file_exists = pathlib.Path(csv_file_path).exists()

if file_exists:
    # Read existing data
    combined_df = pd.read_csv(csv_file_path, parse_dates=['time'])
    if not combined_df.empty:
        # Get the last timestamp
        last_timestamp = combined_df['time'].max()
        # Start from the last timestamp
        start_time = last_timestamp
        print(f"Resuming data fetching from {start_time}.")
    else:
        # If CSV is empty, start from the initial start time
        start_time = initial_start_time
        print(f"The existing CSV file is empty. Starting data fetching from {start_time}.")
else:
    # Initialize an empty DataFrame for combined_df
    combined_df = pd.DataFrame()
    # Start from the initial start time
    start_time = initial_start_time
    print(f"No existing CSV file found. Starting data fetching from {start_time}.")

# End time is the current time
end_time = pd.Timestamp.now(tz='Europe/Berlin')  # Current time in Berlin timezone

# Fetch data in batches
new_data_df = fetch_data_in_batches(start_time, end_time)

# Combine with existing data
if not new_data_df.empty:
    # Append new data to combined_df
    combined_df = pd.concat([combined_df, new_data_df], ignore_index=True)
    # Drop duplicates based on 'time' column
    combined_df.drop_duplicates(subset='time', inplace=True)
    # Save the combined DataFrame to the CSV file
    combined_df.to_csv(csv_file_path, index=False)
    print(f"Data fetching completed and saved to '{csv_file_path}'.")
else:
    print("No new data fetched.")

Resuming data fetching from 2024-11-04 17:28:49.184460+01:00.
Fetching data from 2024-11-04 17:28:49.184460+01:00 to 2024-11-04 22:06:05.834186+01:00
Data fetching completed and saved to '../all_data_files/combined_measurements_data.csv'.


In [7]:
combined_df

,time,correlation_ids_0,device_id,end_device_ids_application_ids_application_id,end_device_ids_dev_addr,end_device_ids_dev_eui,end_device_ids_device_id,end_device_ids_join_eui,received_at,uplink_message_confirmed,...,uplink_message_settings_data_rate_lora_coding_rate,uplink_message_settings_data_rate_lora_spreading_factor,uplink_message_settings_frequency,uplink_message_settings_time,uplink_message_settings_timestamp,uplink_message_version_ids_band_id,uplink_message_version_ids_brand_id,uplink_message_version_ids_firmware_version,uplink_message_version_ids_hardware_version,uplink_message_version_ids_model_id
0,2024-09-26 13:00:52.541686+02:00,gs:uplink:01J8PZVAYFB1Y3CDR10MCHH43E,up,pilot-test,260B46AE,A8610A3230257716,pilotdevice03,0,2024-09-26T11:00:52.510624152Z,True,...,4/5,10.0,867900000,2024-09-26T11:00:52.268982Z,2.627461e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
1,2024-09-26 13:00:52.542462+02:00,gs:uplink:01J8PZVAYFB1Y3CDR10MCHH43E,up,pilot-test,260B46AE,A8610A3230257716,pilotdevice03,0,2024-09-26T11:00:52.510624152Z,True,...,4/5,10.0,867900000,2024-09-26T11:00:52.268982Z,2.627461e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
2,2024-09-26 13:01:08.336258+02:00,gs:uplink:01J8PZVTC2YGT0JW8R7XYW7J9D,up,pilot-test,260B41EC,A8610A34362A8316,pilotdevice,0,2024-09-26T11:01:08.305577574Z,True,...,4/5,9.0,868100000,2024-09-26T11:01:08.065003Z,2.643257e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
3,2024-09-26 13:01:08.338296+02:00,gs:uplink:01J8PZVTC2YGT0JW8R7XYW7J9D,up,pilot-test,260B41EC,A8610A34362A8316,pilotdevice,0,2024-09-26T11:01:08.305577574Z,True,...,4/5,9.0,868100000,2024-09-26T11:01:08.065003Z,2.643257e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
4,2024-09-26 13:01:14.477162+02:00,gs:uplink:01J8PZW0BX58S4QHK38X4A9KHJ,up,pilot-test,260B92C1,A8610A3436385E17,pilotdevice01,0,2024-09-26T11:01:14.446529545Z,True,...,4/5,9.0,867500000,2024-09-26T11:01:14.203163Z,2.649395e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326378,2024-11-04 22:05:22.597292+01:00,gs:uplink:01JBWFY7XW2CCBJKJNHDT7X5MX,up,pilot-test,260B5157,A8610A32301F8516,pilotdevice05,0000000000000000,2024-11-04T21:05:22.570784303Z,True,...,4/5,7.0,867300000,2024-11-04T21:05:22.338273Z,2.586756e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
326379,2024-11-04 22:05:37.781127+01:00,gs:uplink:01JBWFYPRF099QT2Q7BRRNVJV8,up,pilot-test,260B1FF3,A8610A3230257716,pilotdevice03,0000000000000000,2024-11-04T21:05:37.755976453Z,True,...,4/5,9.0,868300000,2024-11-04T21:05:37.521792Z,2.601940e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
326380,2024-11-04 22:05:47.120237+01:00,gs:uplink:01JBWFYZW9C84WH6HF0QTA8066,up,pilot-test,260BED96,A8610A3230378316,pilotdevice02,0000000000000000,2024-11-04T21:05:47.093650406Z,True,...,4/5,8.0,867100000,2024-11-04T21:05:46.864678Z,2.611282e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
326381,2024-11-04 22:05:47.191397+01:00,gs:uplink:01JBWFYZYB7Q6ZRRMFZ4VRHN59,up,pilot-test,260B9807,A8610A3436385E17,pilotdevice01,0000000000000000,2024-11-04T21:05:47.164069332Z,True,...,4/5,9.0,867300000,2024-11-04T21:05:46.926942Z,2.611345e+09,EU_863_870,arduino,1.2.3,1.0,mkr-wan-1310
